# SEQ odd vs even correlation matrix 

This script takes beta values out of the participants during the masks for the odd vs even RSA. 
We then make a correlation matrix where even sequences are on the y- axis, and correlate those with odd sequences on the X axis. We do that for each image (A through F). We expect that the diagonal has high correlation.

In [9]:
$module load pandas

SyntaxError: invalid syntax (<ipython-input-9-9d0a4e636726>, line 1)

In [7]:
import os
from os.path import isfile, exists
import numpy as np
from glob import glob
import pandas as pd
import nibabel as nb
from scipy import stats
from scipy.stats import f
from scipy.stats import norm
from math import sqrt
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio
import nipype.interfaces.freesurfer as fs
import nipype.interfaces.utility as util
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xlrd
Z = norm.ppf

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

ImportError: this version of pandas is incompatible with numpy < 1.12.0
your numpy version is 1.11.3.
Please upgrade numpy to >= 1.12.0 to use this pandas version

In [6]:
#This code grabs the masks and the codes
#sids = ['783143']

sids = ['783125', '783126', '783127', '783128', '783129', '783130', '783131', '783132', '783133', '783134',
        '783135', '783136', '783137', '783138', '783139', '783140', '783141', '783142', '783144', '783146',
        '783147', '783148', '783149', '783150', '783151', '783152', '783153', '783154', '783155', '783156',
        '783157', '783158', '783159', '783163', '783143'] #35 subjects
proj_dir = '/home/data/madlab/data/mri/seqtrd' 
mask_filenames = []
cope_files = []
for SID in sids:
    subjmasks = glob(proj_dir + '/mask/cortical_rois/{0}/anat_masks/_subject_id_{0}/*'.format(SID))
    subjmasks.sort()
    mask_filenames.append(subjmasks)
    #subjcopes = glob(proj_dir + '/frstlvl/seq_item_pos/*_pos*/' + SID + '/modelfit/contrasts/cope02_*_pos*.nii.gz')
    subjcopes = glob(proj_dir + '/frstlvl/odd_vs_even/Inseq1_image*/' + SID + '/modelfit/contrasts/cope0*_inseq1_*_*.nii.gz')
    subjcopes.sort()
    cope_files.append(subjcopes)    
    if len(cope_files[-1]) == 0 or len(cope_files[-1]) != 17:
        print(SID)

783125
783126
783127
783128
783129
783130
783131
783132
783133
783134
783135
783136
783137
783138
783139
783140
783141
783142
783144
783146
783147
783148
783149
783150
783151
783152
783153
783154
783155
783156
783157
783158
783159
783163
783143


## Cell to double-check the array indexing

In [ ]:
#change first index of cope_files to indicate participant index in sids array
for i, curr_mask in enumerate(mask_filenames[34]):
    print(i, mask_filenames[0][i].split('/')[-1][:-7])

In [ ]:
for i, curr_cope in enumerate(cope_files[0]):
    print(i, cope_files[0][i].split('/')[-1][7:-7]) 

### Use binarized mask to obtain activation in left & right hemisphere for each region of interest

In [ ]:
inseq = {'subjid':[],
         'hp_odd_A':[], 'hp_odd_B':[], 'hp_odd_C':[], 'hp_odd_D':[], 'hp_odd_E':[], 'hp_odd_F':[], 
         'hp_even_A':[], 'hp_even_B':[], 'hp_even_C':[], 'hp_even_D':[], 'hp_even_E':[], 'hp_even_F':[],
         'mpfc_odd_A':[], 'mpfc_odd_B':[], 'mpfc_odd_C':[], 'mpfc_odd_D':[], 'mpfc_odd_E':[], 'mpfc_odd_F':[]
         'mpfc_even_A':[], 'mpfc_even_B':[], 'mpfc_even_C':[], 'mpfc_even_D':[], 'mpfc_even_E':[], 'mpfc_even_F':[]
         'caud_odd_A':[], 'caud_odd_B':[], 'caud_odd_C':[], 'caud_odd_D':[], 'caud_odd_E':[],'caud_odd_F':[],
         'caud_even_A':[], 'caud_even_B':[], 'caud_even_C':[], 'caud_even_D':[], 'caud_even_E':[],'caud_even_F':[],
         'suptemp_odd_A':[], 'suptemp_odd_B':[], 'suptemp_odd_C':[], 'suptemp_odd_D':[], 'suptemp_odd_E':[], 'suptemp_odd_F':[],
 
          
for i in range(len(sids)):
    inseq['subjid'].append(sids[i])
    ot['subjid'].append(sids[i])
    repeat['subjid'].append(sids[i])
    skip['subjid'].append(sids[i])
    #import masks
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][7])
    lh_mpfc_img = nb.load(mask_filenames[i][3])
    rh_mpfc_img = nb.load(mask_filenames[i][8])
    lh_caud_img = nb.load(mask_filenames[i][0])
    rh_caud_img = nb.load(mask_filenames[i][5])
    lh_suptemp_img = nb.load(mask_filenames[i][1])
    rh_suptemp_img = nb.load(mask_filenames[i][6])
    #import copes
    inseq_even_A_img = nb.load(cope_files[i][0])
    inseq_even_B_img = nb.load(cope_files[i][1])
    inseq_even_C_img = nb.load(cope_files[i][2])
    inseq_even_D_img = nb.load(cope_files[i][3])
    inseq_even_E_img = nb.load(cope_files[i][4])
    inseq_even_F_img = nb.load(cope_files[i][4])
    inseq_odd_A_img = nb.load(cope_files[i][0])
    inseq_odd_B_img = nb.load(cope_files[i][1])
    inseq_odd_C_img = nb.load(cope_files[i][2])
    inseq_odd_D_img = nb.load(cope_files[i][3])
    inseq_odd_E_img = nb.load(cope_files[i][4])
    inseq_odd_F_img = nb.load(cope_files[i][4])
 

    pos = ['pos2', 'pos3', 'pos4', 'pos5', 'pos6']    
    #flexible loops to populate trial type dictionaries
    for r in ['hp', 'mpfc', 'caud', 'suptemp']:
        for d in ['odd', 'even']:      
         ##########LEFT OFF HERE
            if d in ['skip', 'ot']: #if a skip or ot trial
                for p in pos[:-1]: #iterate through pos2-5
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))   
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    eval('{0}["{1}_{2}"].append(bihemi_data)'.format(d,r,p))
            elif d == 'repeat':#if a repeat trial
                for p in pos[1:]: #iterate through pos3-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    repeat['{0}_{1}'.format(r,p)].append(bihemi_data) 
            else: #if an inseq trial    
                for p in pos: #iterate through pos2-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    inseq['{0}_{1}'.format(r,p)].append(bihemi_data)
#convert dicitionaries to dataframes                    
inseq_df = pd.DataFrame(inseq) 
ot_df = pd.DataFrame(ot) 
repeat_df = pd.DataFrame(repeat) 
skip_df = pd.DataFrame(skip) 

In [ ]:
inseq = {'subjid':[],
         'hp_pos2':[], 'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 'hp_pos6':[],  
         'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[], 'mpfc_pos6':[],
         'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[], 'caud_pos6':[],
         'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[], 'suptemp_pos6':[]}

ot = {'subjid':[],
      'hp_pos2':[], 'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[],  
      'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[],
      'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[],
      'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[]}

repeat = {'subjid':[],
          'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 'hp_pos6':[], 
          'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[], 'mpfc_pos6':[],
          'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[], 'caud_pos6':[],
          'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[], 'suptemp_pos6':[]}

skip = {'subjid':[],
        'hp_pos2':[],'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 
        'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[],
        'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[],
        'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[]}
           
          
for i in range(len(sids)):
    inseq['subjid'].append(sids[i])
    ot['subjid'].append(sids[i])
    repeat['subjid'].append(sids[i])
    skip['subjid'].append(sids[i])
    #import masks
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][7])
    lh_mpfc_img = nb.load(mask_filenames[i][3])
    rh_mpfc_img = nb.load(mask_filenames[i][8])
    lh_caud_img = nb.load(mask_filenames[i][0])
    rh_caud_img = nb.load(mask_filenames[i][5])
    lh_suptemp_img = nb.load(mask_filenames[i][1])
    rh_suptemp_img = nb.load(mask_filenames[i][6])
    #import copes
    inseq_pos2_img = nb.load(cope_files[i][0])
    inseq_pos3_img = nb.load(cope_files[i][1])
    inseq_pos4_img = nb.load(cope_files[i][2])
    inseq_pos5_img = nb.load(cope_files[i][3])
    inseq_pos6_img = nb.load(cope_files[i][4])
    ot_pos2_img = nb.load(cope_files[i][5])
    ot_pos3_img = nb.load(cope_files[i][6])
    ot_pos4_img = nb.load(cope_files[i][7])
    ot_pos5_img = nb.load(cope_files[i][8])
    repeat_pos3_img = nb.load(cope_files[i][9])
    repeat_pos4_img = nb.load(cope_files[i][10])
    repeat_pos5_img = nb.load(cope_files[i][11])
    repeat_pos6_img = nb.load(cope_files[i][12]) 
    skip_pos2_img = nb.load(cope_files[i][13])
    skip_pos3_img = nb.load(cope_files[i][14])
    skip_pos4_img = nb.load(cope_files[i][15])
    skip_pos5_img = nb.load(cope_files[i][16]) 

    pos = ['pos2', 'pos3', 'pos4', 'pos5', 'pos6']    
    #flexible loops to populate trial type dictionaries
    for r in ['hp', 'mpfc', 'caud', 'suptemp']:
        for d in ['inseq', 'ot', 'repeat', 'skip']:      
            if d in ['skip', 'ot']: #if a skip or ot trial
                for p in pos[:-1]: #iterate through pos2-5
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))   
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    eval('{0}["{1}_{2}"].append(bihemi_data)'.format(d,r,p))
            elif d == 'repeat':#if a repeat trial
                for p in pos[1:]: #iterate through pos3-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    repeat['{0}_{1}'.format(r,p)].append(bihemi_data) 
            else: #if an inseq trial    
                for p in pos: #iterate through pos2-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    inseq['{0}_{1}'.format(r,p)].append(bihemi_data)
#convert dicitionaries to dataframes                    
inseq_df = pd.DataFrame(inseq) 
ot_df = pd.DataFrame(ot) 
repeat_df = pd.DataFrame(repeat) 
skip_df = pd.DataFrame(skip) 

In [ ]:
## MPFC -- pos 3, 4, and 5
mpfc_inseq_df = inseq_df.iloc[:,11:14] #all rows, only mpfc_pos3-5 in inseq_df 
mpfc_ot_df = ot_df.iloc[:,9:12] #all rows, only mpfc_pos3-5 in ot_df 
mpfc_repeat_df = repeat_df.iloc[:,8:11] #all rows, only mpfc_pos3-5 in repeat_df 
mpfc_skip_df = skip_df.iloc[:,9:12] #all rows, only mpfc_pos3-5 in skip_df 
#mpfc_skip_df #Checker!!

In [ ]:
## HPC -- pos 3, 4, and 5
hc_inseq_df = inseq_df.iloc[:,6:9]
hc_ot_df = ot_df.iloc[:,5:8]
hc_repeat_df = repeat_df.iloc[:,4:7]
hc_skip_df = skip_df.iloc[:,5:8]
#hc_skip_df

In [2]:
#hc_inseq_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/TEST_HC_Inseq.csv')

In [3]:
## CAUDATE -- pos 3, 4, and 5
caud_inseq_df = inseq_df.iloc[:,1:4]
caud_ot_df = ot_df.iloc[:,1:4]
caud_repeat_df = repeat_df.iloc[:,0:3]
caud_skip_df = skip_df.iloc[:,1:4]
#caud_skip_df

NameError: name 'inseq_df' is not defined

In [4]:
## SUPERIOR TEMPORAL -- pos 3, 4, and 5
suptemp_inseq_df = inseq_df.iloc[:,17:20]
suptemp_ot_df = ot_df.iloc[:,14:18]
suptemp_repeat_df = repeat_df.iloc[:,13:16]
suptemp_skip_df = skip_df.iloc[:,14:17]
#suptemp_skip_df

NameError: name 'inseq_df' is not defined

# HC OT vs InSeq

In [138]:
hc_inseq_df

,hp_pos3,hp_pos4,hp_pos5
0,"[12.3586035, 13.948217, 1.6722425, 18.972544, ...","[16.74321, -15.905314, 5.866419, 29.039343, 12...","[11.079806, -12.52245, -14.535177, -2.9802592,..."
1,"[6.8258266, 1.2097628, 16.716324, 10.889773, 2...","[-4.8534636, 8.406112, -1.3560259, 9.750456, 1...","[-16.783054, 6.6822414, -11.417978, -11.492384..."
2,"[-12.382332, 7.5371776, -4.9155297, 2.7686796,...","[-2.9797509, -4.7135925, -3.507625, -15.710671...","[-16.179743, -25.187332, -16.600986, -1.119875..."
3,"[53.27312, 19.500662, 12.872727, 29.21866, 11....","[47.934467, 23.142508, 9.434773, 27.301834, 11...","[66.623405, 20.242762, 3.5271025, 26.94514, 9...."
4,"[-51.752655, 14.409656, 2.1113021, 7.41068, -2...","[-34.04503, 19.092323, 24.014011, -5.613553, -...","[-45.24395, 5.6748376, 32.53173, -15.530738, -..."
5,"[3.31326, -12.391233, -2.2856026, 11.814476, -...","[2.0325425, -9.055546, -3.5867298, 13.307179, ...","[5.610455, -7.099957, -0.16495456, 4.5027814, ..."
6,"[-5.8845763, -20.411942, 16.606483, 4.126438, ...","[-6.288532, -21.822357, 11.109494, 5.6912904, ...","[-5.8015695, -26.701012, 2.791051, 13.504688, ..."
7,"[-8.6731415, 16.711088, -21.725943, 12.018288,...","[-9.433353, 31.469858, -30.5878, 16.10141, 5.2...","[-9.642869, 9.531282, -22.088356, 10.253892, -..."
8,"[10.921628, 26.351833, 16.867634, 6.4072657, 2...","[29.460546, 9.1470785, 4.4115877, 1.6050123, -...","[38.804577, -17.381369, -2.0189424, -20.126709..."
9,"[-4.1933937, -4.258795, -24.184633, 14.292356,...","[-6.225493, -2.1825883, -19.946592, 11.173768,...","[-5.974105, 6.761761, -13.663758, 15.931409, 5..."


In [139]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,1):
    #This give the correlation value and the p value    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [140]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_OT_pos3', 'p_HC_inseq_OT_pos3', 'r_HC_inseq_OT_pos4', 'p_HC_inseq_OT_pos4', 'r_HC_inseq_OT_pos5', 'p_HC_inseq_OT_pos5', 'r_HC_inseq_pos3_OT_pos4', 'p_HC_inseq_pos3_OT_pos4', 'r_HC_inseq_pos3_OT_pos5', 'p_HC_inseq_pos3_OT_pos5', 'r_HC_inseq_pos4_OT_pos5', 'p_HC_inseq_pos4_OT_pos5', 'r_HC_inseq_pos4_OT_pos3', 'p_HC_inseq_pos4_OT_pos3', 'r_HC_inseq_pos5_OT_pos3', 'p_HC_inseq_pos5_OT_pos3', 'r_HC_inseq_pos5_OT_pos4', 'p_HC_inseq_pos5_OT_pos4']
final_df

,r_HC_inseq_OT_pos3,p_HC_inseq_OT_pos3,r_HC_inseq_OT_pos4,p_HC_inseq_OT_pos4,r_HC_inseq_OT_pos5,p_HC_inseq_OT_pos5,r_HC_inseq_pos3_OT_pos4,p_HC_inseq_pos3_OT_pos4,r_HC_inseq_pos3_OT_pos5,p_HC_inseq_pos3_OT_pos5,r_HC_inseq_pos4_OT_pos5,p_HC_inseq_pos4_OT_pos5,r_HC_inseq_pos4_OT_pos3,p_HC_inseq_pos4_OT_pos3,r_HC_inseq_pos5_OT_pos3,p_HC_inseq_pos5_OT_pos3,r_HC_inseq_pos5_OT_pos4,p_HC_inseq_pos5_OT_pos4
0,0.215995,0.000003,0.226695,8.971789e-07,-0.007117,0.879011,0.3705,2.051512e-16,0.218685,0.000002,0.126486,0.0066,0.480775,5.555082e-28,0.300728,4.536904e-11,0.305871,2.032238e-11


In [141]:
#final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_OT_InSeq_RSA_corr.csv')

# HC Repeat vs InSeq

In [142]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [57]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_repeat_pos3', 'p_HC_inseq_repeat_pos3', 'r_HC_inseq_repeat_pos4', 'p_HC_inseq_repeat_pos4', 'r_HC_inseq_repeat_pos5', 'p_HC_inseq_repeat_pos5', 'r_HC_inseq_pos3_repeat_pos4', 'p_HC_inseq_pos3_repeat_pos4', 'r_HC_inseq_pos3_repeat_pos5', 'p_HC_inseq_pos3_repeat_pos5', 'r_HC_inseq_pos4_repeat_pos5', 'p_HC_inseq_pos4_repeat_pos5', 'r_HC_inseq_pos4_repeat_pos3', 'p_HC_inseq_pos4_repeat_pos3', 'r_HC_inseq_pos5_repeat_pos3', 'p_HC_inseq_pos5_repeat_pos3', 'r_HC_inseq_pos5_repeat_pos4', 'p_HC_inseq_pos5_repeat_pos4']
final_df

,r_HC_inseq_repeat_pos3,p_HC_inseq_repeat_pos3,r_HC_inseq_repeat_pos4,p_HC_inseq_repeat_pos4,r_HC_inseq_repeat_pos5,p_HC_inseq_repeat_pos5,r_HC_inseq_pos3_repeat_pos4,p_HC_inseq_pos3_repeat_pos4,r_HC_inseq_pos3_repeat_pos5,p_HC_inseq_pos3_repeat_pos5,r_HC_inseq_pos4_repeat_pos5,p_HC_inseq_pos4_repeat_pos5,r_HC_inseq_pos4_repeat_pos3,p_HC_inseq_pos4_repeat_pos3,r_HC_inseq_pos5_repeat_pos3,p_HC_inseq_pos5_repeat_pos3,r_HC_inseq_pos5_repeat_pos4,p_HC_inseq_pos5_repeat_pos4
0,0.284647,5.052401e-10,-0.042744,3.603547e-01,0.239028,2.125349e-07,0.142966,2.113948e-03,0.191676,3.499716e-05,0.536336,1.274768e-35,0.428804,5.339604e-22,0.270518,3.713644e-09,0.186626,5.643748e-05
1,0.132585,2.833237e-03,0.398362,1.181459e-20,0.475943,6.534692e-30,0.298858,7.048017e-12,0.324889,7.050114e-14,0.476583,5.352285e-30,0.021355,6.321112e-01,-0.107679,1.548467e-02,0.401705,5.260688e-21
2,0.014772,7.512354e-01,0.171286,2.130063e-04,-0.016074,7.301228e-01,0.253606,3.149459e-08,0.076575,9.983390e-02,0.257395,1.926704e-08,0.198904,1.621593e-05,0.139342,2.657096e-03,0.372986,9.929766e-17
3,0.161284,9.221894e-04,0.195724,5.495615e-05,0.204529,2.456257e-05,0.217478,7.047609e-06,0.278117,7.015976e-09,0.305874,1.591579e-10,0.176551,2.815800e-04,0.107322,2.804760e-02,0.224137,3.598086e-06
4,0.174914,1.115460e-04,0.094778,3.731911e-02,0.116096,1.066440e-02,0.166132,2.452473e-04,0.089269,4.991125e-02,0.194536,1.666102e-05,0.157148,5.276742e-04,0.118971,8.865734e-03,0.286988,1.305395e-10
5,-0.319720,2.714121e-11,0.087182,7.641213e-02,-0.195675,6.118943e-05,0.216307,8.972244e-06,0.184295,1.626890e-04,0.137072,5.209640e-03,0.090439,6.600694e-02,0.111660,2.307564e-02,0.143148,3.513010e-03
6,0.144864,1.379215e-03,0.561409,1.259704e-41,0.293369,4.408234e-11,0.606418,4.912914e-50,0.425200,1.026769e-22,0.485738,4.422003e-30,0.168200,1.982689e-04,0.194344,1.631750e-05,0.680405,3.343915e-67
7,-0.072203,1.145276e-01,0.155357,6.448403e-04,0.162423,3.580171e-04,0.211940,2.873251e-06,0.156847,5.707807e-04,0.366219,1.194289e-16,-0.036663,4.233743e-01,-0.196755,1.441675e-05,0.179931,7.493057e-05
8,-0.162650,1.490378e-04,0.512847,1.728451e-37,0.327186,6.494039e-15,0.563581,1.657768e-46,0.342087,3.048528e-16,0.502824,7.011643e-36,-0.101633,1.826623e-02,-0.147822,5.756285e-04,0.574014,1.464380e-48
9,0.164287,1.704130e-04,0.342988,8.973688e-16,0.027147,5.371886e-01,0.361499,1.816623e-17,0.358440,3.522848e-17,0.419785,1.430129e-23,0.195784,7.020213e-06,0.039606,3.678773e-01,0.547735,5.756738e-42


In [58]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_repeat_InSeq_RSA_corr.csv')

# HC Skip vs InSeq

In [59]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [60]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_skip_pos3', 'p_HC_inseq_skip_pos3', 'r_HC_inseq_skip_pos4', 'p_HC_inseq_skip_pos4', 'r_HC_inseq_skip_pos5', 'p_HC_inseq_skip_pos5', 'r_HC_inseq_pos3_skip_pos4', 'p_HC_inseq_pos3_skip_pos4', 'r_HC_inseq_pos3_skip_pos5', 'p_HC_inseq_pos3_skip_pos5', 'r_HC_inseq_pos4_skip_pos5', 'p_HC_inseq_pos4_skip_pos5', 'r_HC_inseq_pos4_skip_pos3', 'p_HC_inseq_pos4_skip_pos3', 'r_HC_inseq_pos5_skip_pos3', 'p_HC_inseq_pos5_skip_pos3', 'r_HC_inseq_pos5_skip_pos4', 'p_HC_inseq_pos5_skip_pos4']
final_df

,r_HC_inseq_skip_pos3,p_HC_inseq_skip_pos3,r_HC_inseq_skip_pos4,p_HC_inseq_skip_pos4,r_HC_inseq_skip_pos5,p_HC_inseq_skip_pos5,r_HC_inseq_pos3_skip_pos4,p_HC_inseq_pos3_skip_pos4,r_HC_inseq_pos3_skip_pos5,p_HC_inseq_pos3_skip_pos5,r_HC_inseq_pos4_skip_pos5,p_HC_inseq_pos4_skip_pos5,r_HC_inseq_pos4_skip_pos3,p_HC_inseq_pos4_skip_pos3,r_HC_inseq_pos5_skip_pos3,p_HC_inseq_pos5_skip_pos3,r_HC_inseq_pos5_skip_pos4,p_HC_inseq_pos5_skip_pos4
0,0.055259,2.368714e-01,0.253864,3.377378e-08,-0.039241,4.011001e-01,0.390229,3.506279e-18,0.040074,3.911709e-01,0.088262,5.855070e-02,0.262614,1.079237e-08,0.095555,4.050785e-02,0.410900,3.638424e-20
1,0.243231,3.099081e-08,0.322839,1.030138e-13,0.063797,1.522673e-01,0.473939,1.217405e-29,0.130144,3.391258e-03,0.268548,8.627780e-10,0.177099,6.291276e-05,-0.045262,3.100415e-01,0.375757,2.216432e-18
2,-0.039891,3.917915e-01,0.134660,3.696740e-03,0.091195,4.987254e-02,0.328192,4.344117e-13,0.026528,5.691086e-01,0.181737,8.398024e-05,0.435902,6.777502e-23,0.286866,3.203952e-10,0.279106,9.883811e-10
3,0.470226,1.941934e-24,0.580896,3.555997e-39,0.236595,9.670926e-07,0.601886,1.195302e-42,0.281235,4.679399e-09,0.287587,2.018593e-09,0.532547,4.590021e-32,0.520128,2.038818e-30,0.601648,1.313228e-42
4,0.346566,4.464041e-15,0.613342,3.032089e-51,0.043056,3.450468e-01,0.660559,7.236161e-62,-0.125913,5.586926e-03,0.018476,6.854488e-01,0.510713,1.919651e-33,0.477243,7.642830e-29,0.691504,6.080866e-70
5,0.076826,1.185790e-01,0.045241,3.585156e-01,0.161060,1.006620e-03,0.172575,4.198726e-04,-0.109855,2.540207e-02,0.103938,3.450137e-02,0.206570,2.273930e-05,-0.033011,5.029634e-01,0.094958,5.352964e-02
6,0.534209,3.882521e-37,0.075621,9.621968e-02,0.160505,3.872001e-04,0.164346,2.782436e-04,0.169738,1.728376e-04,0.217378,1.346253e-06,0.611071,5.546785e-51,0.612023,3.533488e-51,0.121745,7.269933e-03
7,0.313978,2.030022e-12,0.000188,9.967330e-01,-0.090344,4.813703e-02,0.217285,1.582149e-06,0.087147,5.665660e-02,0.020678,6.516851e-01,0.473629,3.738314e-28,0.385445,2.052169e-18,0.093495,4.081839e-02
8,-0.135118,1.665953e-03,0.427180,2.558615e-25,0.260505,8.237731e-10,0.458202,2.471940e-29,0.187115,1.227093e-05,0.378902,7.616866e-20,0.177654,3.354848e-05,0.044861,2.985167e-01,0.504578,3.700708e-36
9,0.132781,2.436698e-03,0.298429,3.891401e-12,0.092217,3.570631e-02,0.361835,1.688197e-17,0.110881,1.147979e-02,0.106190,1.551277e-02,0.256611,3.001416e-09,-0.052714,2.305836e-01,0.397030,4.829460e-21


In [61]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_skip_InSeq_RSA_corr.csv')

# mPFC correlations

# mpfc OT vs InSeq

In [62]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [63]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_OT_pos3', 'p_mpfc_inseq_OT_pos3', 'r_mpfc_inseq_OT_pos4', 'p_mpfc_inseq_OT_pos4', 'r_mpfc_inseq_OT_pos5', 'p_mpfc_inseq_OT_pos5', 'r_mpfc_inseq_pos3_OT_pos4', 'p_mpfc_inseq_pos3_OT_pos4', 'r_mpfc_inseq_pos3_OT_pos5', 'p_mpfc_inseq_pos3_OT_pos5', 'r_mpfc_inseq_pos4_OT_pos5', 'p_mpfc_inseq_pos4_OT_pos5', 'r_mpfc_inseq_pos4_OT_pos3', 'p_mpfc_inseq_pos4_OT_pos3', 'r_mpfc_inseq_pos5_OT_pos3', 'p_mpfc_inseq_pos5_OT_pos3', 'r_mpfc_inseq_pos5_OT_pos4', 'p_mpfc_inseq_pos5_OT_pos4']
final_df

,r_mpfc_inseq_OT_pos3,p_mpfc_inseq_OT_pos3,r_mpfc_inseq_OT_pos4,p_mpfc_inseq_OT_pos4,r_mpfc_inseq_OT_pos5,p_mpfc_inseq_OT_pos5,r_mpfc_inseq_pos3_OT_pos4,p_mpfc_inseq_pos3_OT_pos4,r_mpfc_inseq_pos3_OT_pos5,p_mpfc_inseq_pos3_OT_pos5,r_mpfc_inseq_pos4_OT_pos5,p_mpfc_inseq_pos4_OT_pos5,r_mpfc_inseq_pos4_OT_pos3,p_mpfc_inseq_pos4_OT_pos3,r_mpfc_inseq_pos5_OT_pos3,p_mpfc_inseq_pos5_OT_pos3,r_mpfc_inseq_pos5_OT_pos4,p_mpfc_inseq_pos5_OT_pos4
0,0.085797,4.903451e-07,0.522815,8.925709e-240,-0.057461,7.644047e-04,0.622225,0.000000e+00,0.261566,1.016721e-54,0.036553,3.237131e-02,0.240404,2.995604e-46,0.196072,4.817143e-31,0.513587,4.818766e-230
1,-0.048586,1.355417e-03,0.221694,1.570870e-49,-0.016687,2.713970e-01,0.118005,5.978048e-15,0.142420,3.934201e-21,0.286912,3.899803e-83,0.025799,8.902630e-02,-0.011801,4.367212e-01,0.229827,3.395482e-53
2,-0.023116,1.772791e-01,0.369659,7.853797e-111,0.181548,1.216631e-26,0.581663,8.332554e-308,0.497136,3.012224e-212,0.392993,2.915764e-126,0.366957,3.997876e-109,0.344007,2.789140e-95,0.287819,5.238905e-66
3,0.550246,5.318887e-274,0.662526,0.000000e+00,0.029496,8.229673e-02,0.676373,0.000000e+00,0.121534,6.756792e-13,0.122215,5.018342e-13,0.587366,4.308252e-321,0.549183,9.739213e-273,0.660284,0.000000e+00
4,0.415700,3.929847e-140,0.654691,0.000000e+00,0.449488,2.528394e-166,0.689804,0.000000e+00,0.573731,1.359283e-292,0.572149,1.246837e-290,0.519944,1.927687e-231,0.483124,1.639363e-195,0.641476,0.000000e+00
5,0.108166,1.169794e-10,0.044228,8.586845e-03,-0.158211,3.192802e-21,0.376040,5.669595e-119,0.203100,3.517220e-34,0.058134,5.489715e-04,0.143062,1.332950e-17,-0.067569,5.876490e-05,0.229854,1.517235e-43
6,0.207120,5.231841e-38,0.511269,1.287664e-251,0.132643,2.393656e-16,0.576934,0.000000e+00,0.289774,3.078903e-74,0.279163,8.103317e-69,0.253118,1.695441e-56,0.186027,7.288381e-31,0.538435,4.916526e-284
7,0.054192,1.181752e-03,0.264961,1.450233e-58,0.049452,3.084322e-03,0.311629,1.910469e-81,0.307757,2.221928e-79,0.234373,7.411209e-46,0.213200,4.575366e-38,0.270575,4.333401e-61,0.284294,1.619702e-67
8,0.027506,9.137819e-02,0.211209,2.936457e-39,0.265836,5.661479e-62,0.382749,9.773053e-132,0.279490,1.421684e-68,0.399202,3.552026e-144,0.014329,3.792206e-01,-0.004882,7.645162e-01,0.337704,3.644635e-101
9,0.486309,1.051031e-211,0.402589,1.926463e-139,0.232507,4.221597e-45,0.539604,2.034851e-269,0.292517,1.725370e-71,0.294691,1.410769e-72,0.580004,1.268761e-320,0.578847,4.674503e-319,0.519338,2.976618e-246


In [64]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_OT_InSeq_RSA_corr.csv')

# mpfc repeat vs InSeq

In [65]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [66]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_repeat_pos3', 'p_mpfc_inseq_repeat_pos3', 'r_mpfc_inseq_repeat_pos4', 'p_mpfc_inseq_repeat_pos4', 'r_mpfc_inseq_repeat_pos5', 'p_mpfc_inseq_repeat_pos5', 'r_mpfc_inseq_pos3_repeat_pos4', 'p_mpfc_inseq_pos3_repeat_pos4', 'r_mpfc_inseq_pos3_repeat_pos5', 'p_mpfc_inseq_pos3_repeat_pos5', 'r_mpfc_inseq_pos4_repeat_pos5', 'p_mpfc_inseq_pos4_repeat_pos5', 'r_mpfc_inseq_pos4_repeat_pos3', 'p_mpfc_inseq_pos4_repeat_pos3', 'r_mpfc_inseq_pos5_repeat_pos3', 'p_mpfc_inseq_pos5_repeat_pos3', 'r_mpfc_inseq_pos5_repeat_pos4', 'p_mpfc_inseq_pos5_repeat_pos4']
final_df

,r_mpfc_inseq_repeat_pos3,p_mpfc_inseq_repeat_pos3,r_mpfc_inseq_repeat_pos4,p_mpfc_inseq_repeat_pos4,r_mpfc_inseq_repeat_pos5,p_mpfc_inseq_repeat_pos5,r_mpfc_inseq_pos3_repeat_pos4,p_mpfc_inseq_pos3_repeat_pos4,r_mpfc_inseq_pos3_repeat_pos5,p_mpfc_inseq_pos3_repeat_pos5,r_mpfc_inseq_pos4_repeat_pos5,p_mpfc_inseq_pos4_repeat_pos5,r_mpfc_inseq_pos4_repeat_pos3,p_mpfc_inseq_pos4_repeat_pos3,r_mpfc_inseq_pos5_repeat_pos3,p_mpfc_inseq_pos5_repeat_pos3,r_mpfc_inseq_pos5_repeat_pos4,p_mpfc_inseq_pos5_repeat_pos4
0,0.173821,1.183309e-24,-0.154890,7.572863e-20,0.207082,1.653618e-34,0.058504,6.113266e-04,0.006196,7.168921e-01,0.310896,1.094454e-77,0.263967,9.897499e-56,0.221535,2.290195e-39,0.043910,1.014554e-02
1,0.003173,8.343672e-01,0.202096,2.751760e-41,0.003601,8.124135e-01,0.193721,5.155900e-38,0.137208,1.030131e-19,0.263218,8.562745e-70,-0.023598,1.198323e-01,-0.073928,1.066746e-06,0.104649,4.647885e-12
2,0.235019,5.490969e-44,0.349947,9.376226e-99,0.056505,9.664895e-04,0.359545,1.580326e-104,0.344274,1.954617e-95,0.392172,1.070258e-125,0.270630,2.796163e-58,0.116903,7.617140e-12,0.448011,6.111466e-168
3,0.222463,3.533685e-40,0.248631,4.644502e-50,0.343968,5.515717e-97,0.267798,4.414389e-58,0.385799,1.406839e-123,0.411527,5.542286e-142,0.261103,3.342239e-55,0.281876,2.074615e-64,0.267058,9.271919e-58
4,0.308617,7.623422e-75,0.146907,1.273041e-17,0.174661,2.341282e-24,0.160437,9.357925e-21,0.073400,2.114399e-05,0.207489,6.716467e-34,0.351867,3.171584e-98,0.304634,7.046647e-73,0.212600,1.526571e-35
5,-0.094837,1.648358e-08,-0.039799,1.804369e-02,-0.174889,1.213031e-25,0.374943,3.090257e-118,0.129823,9.684067e-15,0.054255,1.260835e-03,-0.037864,2.447006e-02,-0.052650,1.753001e-03,0.067807,5.532905e-05
6,0.515534,1.634768e-256,0.653187,0.000000e+00,0.435800,1.505716e-175,0.658826,0.000000e+00,0.535516,2.080440e-280,0.544176,2.845566e-291,0.533073,2.118253e-277,0.496038,1.091000e-234,0.636500,0.000000e+00
7,0.179837,2.141587e-27,0.317634,1.035485e-84,0.292681,1.250012e-71,0.324641,1.283003e-88,0.189883,2.092108e-30,0.351013,2.802929e-104,0.278935,5.804338e-65,0.209074,1.216831e-36,0.270880,3.145150e-61
8,-0.296308,3.106415e-77,0.442923,7.586202e-181,0.496692,5.560516e-234,0.526672,5.191243e-268,0.522354,6.748177e-263,0.587777,0.000000e+00,-0.282664,3.662018e-70,-0.223253,8.966290e-44,0.542183,5.271322e-287
9,0.230640,2.178530e-44,0.518923,8.537625e-246,0.358203,9.951318e-109,0.498806,2.478647e-224,0.358770,4.315298e-109,0.422101,1.641619e-154,0.305137,6.176922e-78,0.257622,2.603715e-55,0.606401,0.000000e+00


In [67]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_repeat_InSeq_RSA_corr.csv')

# mpfc Skip vs InSeq

In [68]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [69]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_skip_pos3', 'p_mpfc_inseq_skip_pos3', 'r_mpfc_inseq_skip_pos4', 'p_mpfc_inseq_skip_pos4', 'r_mpfc_inseq_skip_pos5', 'p_mpfc_inseq_skip_pos5', 'r_mpfc_inseq_pos3_skip_pos4', 'p_mpfc_inseq_pos3_skip_pos4', 'r_mpfc_inseq_pos3_skip_pos5', 'p_mpfc_inseq_pos3_skip_pos5', 'r_mpfc_inseq_pos4_skip_pos5', 'p_mpfc_inseq_pos4_skip_pos5', 'r_mpfc_inseq_pos4_skip_pos3', 'p_mpfc_inseq_pos4_skip_pos3', 'r_mpfc_inseq_pos5_skip_pos3', 'p_mpfc_inseq_pos5_skip_pos3', 'r_mpfc_inseq_pos5_skip_pos4', 'p_mpfc_inseq_pos5_skip_pos4']
final_df

,r_mpfc_inseq_skip_pos3,p_mpfc_inseq_skip_pos3,r_mpfc_inseq_skip_pos4,p_mpfc_inseq_skip_pos4,r_mpfc_inseq_skip_pos5,p_mpfc_inseq_skip_pos5,r_mpfc_inseq_pos3_skip_pos4,p_mpfc_inseq_pos3_skip_pos4,r_mpfc_inseq_pos3_skip_pos5,p_mpfc_inseq_pos3_skip_pos5,r_mpfc_inseq_pos4_skip_pos5,p_mpfc_inseq_pos4_skip_pos5,r_mpfc_inseq_pos4_skip_pos3,p_mpfc_inseq_pos4_skip_pos3,r_mpfc_inseq_pos5_skip_pos3,p_mpfc_inseq_pos5_skip_pos3,r_mpfc_inseq_pos5_skip_pos4,p_mpfc_inseq_pos5_skip_pos4
0,0.241022,1.739883e-46,0.108051,2.273794e-10,0.249584,7.966052e-50,0.318251,1.620111e-81,0.273271,9.415402e-60,0.361398,3.033993e-106,0.398755,5.511384e-131,0.218081,3.576928e-38,0.251260,1.707187e-50
1,0.279463,8.497723e-79,0.102680,1.160188e-11,-0.064490,2.093870e-05,0.221963,1.194607e-49,0.021189,1.625242e-01,0.148596,6.994813e-23,0.209300,3.203592e-44,0.021018,1.659519e-01,0.078606,2.119087e-07
2,0.123528,4.606762e-13,0.227873,2.178257e-41,0.096332,1.753528e-08,0.306445,5.155274e-75,0.258248,4.726897e-53,0.303276,1.960570e-73,0.472205,7.413260e-189,0.450166,9.796555e-170,0.419678,1.659172e-145
3,0.537871,1.397254e-259,0.646182,0.000000e+00,0.345488,6.971551e-98,0.645818,0.000000e+00,0.361512,1.176773e-107,0.360107,8.922251e-107,0.586717,3.240083e-320,0.563199,9.351626e-290,0.690362,0.000000e+00
4,0.544417,6.865741e-258,0.670429,0.000000e+00,0.041199,1.709384e-02,0.684964,0.000000e+00,-0.067044,1.030247e-04,0.043081,1.264146e-02,0.650674,0.000000e+00,0.528167,4.364335e-240,0.688307,0.000000e+00
5,-0.153960,3.613481e-20,-0.016949,3.140661e-01,0.081062,1.418295e-06,0.021489,2.017907e-01,-0.061014,2.865574e-04,0.136390,3.997611e-16,0.111159,3.529720e-11,-0.047045,5.178797e-03,0.167204,1.505888e-23
6,0.492425,8.509730e-231,0.525579,2.516527e-268,0.273130,7.690274e-66,0.560050,4.923488e-312,0.258983,3.758438e-59,0.288127,2.212703e-73,0.579855,0.000000e+00,0.469134,8.374050e-207,0.548205,1.946930e-296
7,0.184665,8.045626e-29,-0.073766,9.984833e-06,-0.008887,5.950640e-01,0.098878,3.070353e-09,0.256400,7.854982e-55,0.193693,1.359413e-31,0.171069,6.575407e-25,-0.002438,8.840895e-01,-0.119566,7.163506e-13
8,0.258749,1.070519e-58,0.336404,2.354913e-100,0.403684,1.090359e-147,0.403958,6.627148e-148,0.374782,5.769748e-126,0.548462,5.577000e-295,0.498452,6.912501e-236,0.361855,6.112466e-117,0.446346,5.954192e-184
9,0.225377,2.054940e-42,0.252123,5.643233e-53,0.326099,2.267776e-89,0.334244,4.569402e-94,0.173925,1.102235e-25,0.285016,8.246478e-68,0.394336,2.298477e-133,0.222729,1.937690e-41,0.367079,1.714827e-114


In [70]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_skip_InSeq_RSA_corr.csv')

#    CAUDATE 

# caudate OT vs InSeq

In [71]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation (r) value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [72]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
#Adding column names
final_df.columns = ['r_caud_inseq_OT_pos3', 'p_caud_inseq_OT_pos3', 'r_caud_inseq_OT_pos4', 'p_caud_inseq_OT_pos4', 'r_caud_inseq_OT_pos5', 'p_caud_inseq_OT_pos5', 'r_caud_inseq_pos3_OT_pos4', 'p_caud_inseq_pos3_OT_pos4', 'r_caud_inseq_pos3_OT_pos5', 'p_caud_inseq_pos3_OT_pos5', 'r_caud_inseq_pos4_OT_pos5', 'p_caud_inseq_pos4_OT_pos5', 'r_caud_inseq_pos4_OT_pos3', 'p_caud_inseq_pos4_OT_pos3', 'r_caud_inseq_pos5_OT_pos3', 'p_caud_inseq_pos5_OT_pos3', 'r_caud_inseq_pos5_OT_pos4', 'p_caud_inseq_pos5_OT_pos4']
final_df

,r_caud_inseq_OT_pos3,p_caud_inseq_OT_pos3,r_caud_inseq_OT_pos4,p_caud_inseq_OT_pos4,r_caud_inseq_OT_pos5,p_caud_inseq_OT_pos5,r_caud_inseq_pos3_OT_pos4,p_caud_inseq_pos3_OT_pos4,r_caud_inseq_pos3_OT_pos5,p_caud_inseq_pos3_OT_pos5,r_caud_inseq_pos4_OT_pos5,p_caud_inseq_pos4_OT_pos5,r_caud_inseq_pos4_OT_pos3,p_caud_inseq_pos4_OT_pos3,r_caud_inseq_pos5_OT_pos3,p_caud_inseq_pos5_OT_pos3,r_caud_inseq_pos5_OT_pos4,p_caud_inseq_pos5_OT_pos4
0,0.121329,1.443675e-02,0.315531,7.764829e-11,0.290279,2.526664e-09,0.634830,3.465364e-47,0.340517,1.764775e-12,0.323039,2.582525e-11,0.221644,6.530647e-06,0.186778,1.535707e-04,0.565995,9.226359e-36
1,-0.010561,8.249621e-01,0.077696,1.032202e-01,-0.124086,9.094167e-03,0.138183,3.642525e-03,-0.020304,6.706853e-01,0.041464,3.850447e-01,0.030475,5.232788e-01,-0.023372,6.244980e-01,0.202261,1.871049e-05
2,-0.151018,2.280341e-03,0.188576,1.321923e-04,-0.053888,2.786991e-01,0.476837,1.933911e-24,0.449117,1.509687e-21,0.273893,2.030286e-08,0.249670,3.466687e-07,0.226253,4.131513e-06,0.225492,4.458856e-06
3,0.478084,5.233676e-24,0.667875,1.739810e-52,0.047855,3.421975e-01,0.714679,3.832079e-63,0.180049,3.169144e-04,0.150376,2.699209e-03,0.593772,4.303991e-39,0.616557,7.828792e-43,0.642467,1.781014e-47
4,0.232789,1.783544e-06,0.528035,5.871175e-31,0.396285,6.053943e-17,0.576433,7.497227e-38,0.536924,3.847808e-32,0.567817,1.535924e-36,0.363765,2.460229e-14,0.400503,2.641772e-17,0.546469,1.881923e-33
5,0.093498,7.441437e-02,0.359206,1.476893e-12,0.235336,5.502411e-06,0.501702,1.153414e-24,0.297171,7.069078e-09,0.295484,8.671289e-09,0.304315,2.932395e-09,0.113324,3.041752e-02,0.346327,1.006079e-11
6,0.291295,1.040478e-08,0.372918,1.019066e-13,0.148895,3.999838e-03,0.528812,3.448855e-28,0.263927,2.402164e-07,0.267587,1.609983e-07,0.332930,4.439783e-11,0.334691,3.458214e-11,0.445705,1.481621e-19
7,0.108857,2.929256e-02,0.399497,8.498397e-17,0.187860,1.543942e-04,0.525265,8.079902e-30,0.299019,9.994050e-10,0.287500,4.533431e-09,0.203254,4.125711e-05,0.168424,7.082052e-04,0.390826,4.387456e-16
8,0.164771,9.688513e-04,0.446626,6.558874e-21,0.122380,1.456663e-02,0.631482,1.124625e-45,0.056836,2.579602e-01,0.255575,2.356563e-07,0.247929,5.470492e-07,0.197933,7.016140e-05,0.544881,3.749688e-32
9,0.394567,4.159371e-19,0.342427,1.739835e-14,0.415821,3.050441e-21,0.449825,5.455601e-25,0.324149,4.660176e-13,0.379454,1.111292e-17,0.497626,5.042449e-31,0.294963,5.709364e-11,0.404133,4.755520e-20


In [73]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_OT_InSeq_RSA_corr.csv')

# caudate repeat vs InSeq

In [74]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [75]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_caud_inseq_repeat_pos3', 'p_caud_inseq_repeat_pos3', 'r_caud_inseq_repeat_pos4', 'p_caud_inseq_repeat_pos4', 'r_caud_inseq_repeat_pos5', 'p_caud_inseq_repeat_pos5', 'r_caud_inseq_pos3_repeat_pos4', 'p_caud_inseq_pos3_repeat_pos4', 'r_caud_inseq_pos3_repeat_pos5', 'p_caud_inseq_pos3_repeat_pos5', 'r_caud_inseq_pos4_repeat_pos5', 'p_caud_inseq_pos4_repeat_pos5', 'r_caud_inseq_pos4_repeat_pos3', 'p_caud_inseq_pos4_repeat_pos3', 'r_caud_inseq_pos5_repeat_pos3', 'p_caud_inseq_pos5_repeat_pos3', 'r_caud_inseq_pos5_repeat_pos4', 'p_caud_inseq_pos5_repeat_pos4']
final_df

,r_caud_inseq_repeat_pos3,p_caud_inseq_repeat_pos3,r_caud_inseq_repeat_pos4,p_caud_inseq_repeat_pos4,r_caud_inseq_repeat_pos5,p_caud_inseq_repeat_pos5,r_caud_inseq_pos3_repeat_pos4,p_caud_inseq_pos3_repeat_pos4,r_caud_inseq_pos3_repeat_pos5,p_caud_inseq_pos3_repeat_pos5,r_caud_inseq_pos4_repeat_pos5,p_caud_inseq_pos4_repeat_pos5,r_caud_inseq_pos4_repeat_pos3,p_caud_inseq_pos4_repeat_pos3,r_caud_inseq_pos5_repeat_pos3,p_caud_inseq_pos5_repeat_pos3,r_caud_inseq_pos5_repeat_pos4,p_caud_inseq_pos5_repeat_pos4
0,0.322127,2.956513e-11,-0.020081,6.866363e-01,0.057602,2.468531e-01,0.129265,9.119470e-03,0.115782,1.961664e-02,0.386972,5.937383e-16,0.489448,7.596851e-26,0.346354,6.931616e-13,0.185774,1.668741e-04
1,-0.229228,1.144301e-06,0.043402,3.631973e-01,0.301006,1.093352e-10,0.152712,1.296587e-03,0.240735,3.112214e-07,0.253459,6.818417e-08,-0.156388,9.838186e-04,-0.299608,1.344769e-10,0.255325,5.419337e-08
2,0.135330,6.314348e-03,0.209010,2.181828e-05,-0.063670,2.004553e-01,0.364657,3.252990e-14,0.256348,1.631033e-07,0.351587,2.949268e-13,0.165345,8.246453e-04,0.040506,4.156536e-01,0.239822,1.013990e-06
3,0.286008,6.832324e-09,0.228823,4.218648e-06,0.328227,2.121056e-11,0.223679,6.984703e-06,0.366372,5.030808e-14,0.416267,5.002834e-18,0.337563,5.209326e-12,0.348497,9.464799e-13,0.263857,9.885633e-08
4,0.089837,6.850988e-02,-0.081828,9.718312e-02,0.033086,5.030347e-01,-0.028946,5.579549e-01,-0.002663,9.570201e-01,0.086350,8.000289e-02,0.138355,4.903598e-03,0.060700,2.188988e-01,0.126545,1.013714e-02
5,-0.026779,6.100910e-01,0.061768,2.391380e-01,0.128319,1.415667e-02,0.050767,3.334465e-01,0.265878,2.532977e-07,0.329975,1.016408e-10,0.051110,3.301891e-01,0.082339,1.163304e-01,0.237283,4.575816e-06
6,0.197013,1.309802e-04,0.488339,1.088259e-23,0.197673,1.241979e-04,0.548402,1.359510e-30,0.389620,6.200846e-15,0.445625,1.506574e-19,0.267024,1.712977e-07,0.237258,3.711685e-06,0.569335,2.415676e-33
7,-0.026220,6.006168e-01,0.238904,1.302549e-06,0.235494,1.858129e-06,0.367705,2.773251e-14,0.227776,4.071872e-06,0.341660,2.019494e-12,0.112912,2.374421e-02,0.079422,1.122956e-01,0.203430,4.061490e-05
8,0.332672,9.700696e-12,0.264310,8.705576e-08,0.340780,2.800321e-12,0.381068,3.337928e-15,0.462633,1.675408e-22,0.479426,2.887358e-24,0.228390,4.164262e-06,0.228044,4.310085e-06,0.354498,3.145884e-13
9,0.089739,5.087391e-02,0.353617,2.082692e-15,0.208906,4.498904e-06,0.368085,1.178249e-16,0.305371,1.092232e-11,0.387762,1.864747e-18,0.142675,1.845638e-03,0.058056,2.070554e-01,0.468596,3.019560e-27


In [76]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_repeat_InSeq_RSA_corr.csv')

# caudate Skip vs InSeq

In [77]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [78]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_caud_inseq_skip_pos3', 'p_caud_inseq_skip_pos3', 'r_caud_inseq_skip_pos4', 'p_caud_inseq_skip_pos4', 'r_caud_inseq_skip_pos5', 'p_caud_inseq_skip_pos5', 'r_caud_inseq_pos3_skip_pos4', 'p_caud_inseq_pos3_skip_pos4', 'r_caud_inseq_pos3_skip_pos5', 'p_caud_inseq_pos3_skip_pos5', 'r_caud_inseq_pos4_skip_pos5', 'p_caud_inseq_pos4_skip_pos5', 'r_caud_inseq_pos4_skip_pos3', 'p_caud_inseq_pos4_skip_pos3', 'r_caud_inseq_pos5_skip_pos3', 'p_caud_inseq_pos5_skip_pos3', 'r_caud_inseq_pos5_skip_pos4', 'p_caud_inseq_pos5_skip_pos4']
final_df

,r_caud_inseq_skip_pos3,p_caud_inseq_skip_pos3,r_caud_inseq_skip_pos4,p_caud_inseq_skip_pos4,r_caud_inseq_skip_pos5,p_caud_inseq_skip_pos5,r_caud_inseq_pos3_skip_pos4,p_caud_inseq_pos3_skip_pos4,r_caud_inseq_pos3_skip_pos5,p_caud_inseq_pos3_skip_pos5,r_caud_inseq_pos4_skip_pos5,p_caud_inseq_pos4_skip_pos5,r_caud_inseq_pos4_skip_pos3,p_caud_inseq_pos4_skip_pos3,r_caud_inseq_pos5_skip_pos3,p_caud_inseq_pos5_skip_pos3,r_caud_inseq_pos5_skip_pos4,p_caud_inseq_pos5_skip_pos4
0,-0.292170,1.969249e-09,0.247077,4.619403e-07,0.202682,3.886678e-05,0.400276,4.699358e-17,0.215601,1.173275e-05,0.210900,1.829774e-05,-0.042065,3.979150e-01,-0.182141,2.245809e-04,0.315757,7.515059e-11
1,-0.104411,2.835056e-02,0.253903,6.456656e-08,0.081283,8.821103e-02,0.359962,6.152911e-15,0.176876,1.889659e-04,0.174565,2.297945e-04,0.038124,4.245073e-01,0.006143,8.976391e-01,0.468422,1.959245e-25
2,0.045475,3.607471e-01,0.282472,6.935005e-09,-0.013142,7.917866e-01,0.399974,4.984208e-17,0.064950,1.915385e-01,0.093617,5.947728e-02,0.414933,2.510107e-18,0.287601,3.584881e-09,0.209996,1.990909e-05
3,0.296730,1.718971e-09,0.606881,3.303578e-41,0.125836,1.220519e-02,0.566230,5.960315e-35,0.169770,6.929164e-04,0.192758,1.134000e-04,0.406508,3.437396e-17,0.413043,9.523956e-18,0.652422,2.213281e-49
4,0.311226,1.053150e-10,0.642014,3.026605e-49,-0.049791,3.133613e-01,0.706093,1.945085e-63,-0.100987,4.047867e-02,-0.010108,8.379148e-01,0.476559,9.504516e-25,0.430247,5.395679e-20,0.712513,4.499456e-65
5,0.013177,8.019032e-01,0.140360,7.238137e-03,0.014388,7.841136e-01,0.332205,7.473854e-11,-0.087034,9.686759e-02,0.055878,2.870058e-01,0.021805,6.779986e-01,-0.134590,1.004626e-02,0.211488,4.645716e-05
6,0.414752,6.730652e-17,0.338056,2.135926e-11,0.159788,1.992150e-03,0.468920,9.810343e-22,0.131853,1.090726e-02,0.171531,8.941294e-04,0.557795,8.381810e-32,0.477702,1.327446e-22,0.423969,1.161689e-17
7,0.229429,3.449847e-06,0.164719,9.301411e-04,0.098466,4.879154e-02,0.252310,3.058534e-07,0.255556,2.126461e-07,0.311217,1.871032e-10,0.167983,7.317850e-04,0.021184,6.723380e-01,0.185599,1.858279e-04
8,-0.087123,8.257550e-02,0.199069,6.360759e-05,0.143961,4.002309e-03,0.308096,3.372895e-10,0.124010,1.329502e-02,0.287973,4.875053e-09,0.173520,5.066137e-04,-0.005145,9.185049e-01,0.302945,6.815008e-10
9,0.288318,1.585710e-10,0.196587,1.626453e-05,0.047459,3.024855e-01,0.266705,3.670217e-09,0.097889,3.311425e-02,0.117216,1.064824e-02,0.417523,2.026267e-21,0.107545,1.917886e-02,0.256366,1.499341e-08


In [79]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_skip_InSeq_RSA_corr.csv')

# Superior Temporal Gyrus

### Now we will do a RSA to compare OutSeq and InSeq on pos 3, 4, and 5. We will do this for all the 3 different trials (OT/repeat/skip), and we use the superior temporal gyrus as the mask. 

### RSA of Ordinal Transfers vs. InSeq in the Superior Temporal Gyrus

In [80]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation (r) value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [81]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
#Adding column names
final_df.columns = ['r_suptemp_inseq_OT_pos3', 'p_suptemp_inseq_OT_pos3', 'r_suptemp_inseq_OT_pos4', 'p_suptemp_inseq_OT_pos4', 'r_suptemp_inseq_OT_pos5', 'p_suptemp_inseq_OT_pos5', 'r_suptemp_inseq_pos3_OT_pos4', 'p_suptemp_inseq_pos3_OT_pos4', 'r_suptemp_inseq_pos3_OT_pos5', 'p_suptemp_inseq_pos3_OT_pos5', 'r_suptemp_inseq_pos4_OT_pos5', 'p_suptemp_inseq_pos4_OT_pos5', 'r_suptemp_inseq_pos4_OT_pos3', 'p_suptemp_inseq_pos4_OT_pos3', 'r_suptemp_inseq_pos5_OT_pos3', 'p_suptemp_inseq_pos5_OT_pos3', 'r_suptemp_inseq_pos5_OT_pos4', 'p_suptemp_inseq_pos5_OT_pos4']
final_df

,r_suptemp_inseq_OT_pos3,p_suptemp_inseq_OT_pos3,r_suptemp_inseq_OT_pos4,p_suptemp_inseq_OT_pos4,r_suptemp_inseq_OT_pos5,p_suptemp_inseq_OT_pos5,r_suptemp_inseq_pos3_OT_pos4,p_suptemp_inseq_pos3_OT_pos4,r_suptemp_inseq_pos3_OT_pos5,p_suptemp_inseq_pos3_OT_pos5,r_suptemp_inseq_pos4_OT_pos5,p_suptemp_inseq_pos4_OT_pos5,r_suptemp_inseq_pos4_OT_pos3,p_suptemp_inseq_pos4_OT_pos3,r_suptemp_inseq_pos5_OT_pos3,p_suptemp_inseq_pos5_OT_pos3,r_suptemp_inseq_pos5_OT_pos4,p_suptemp_inseq_pos5_OT_pos4
0,-0.073932,1.199681e-02,0.533313,8.417488e-86,0.011054,7.075861e-01,0.664759,4.490574e-148,0.377173,2.543621e-40,0.093945,1.398363e-03,0.243913,4.309690e-17,0.170584,5.498114e-09,0.575838,6.841669e-103
1,-0.022466,4.108825e-01,0.168047,5.847844e-10,-0.047717,8.057040e-02,0.240753,3.779762e-19,0.039240,1.507968e-01,0.106465,9.319646e-05,-0.006420,8.142408e-01,0.030656,2.617605e-01,0.090024,9.616800e-04
2,0.033831,2.248307e-01,0.377732,5.557776e-45,0.156891,1.492316e-08,0.431162,1.705008e-59,0.331869,1.632573e-34,0.295223,2.445574e-27,0.322902,1.154806e-32,0.366574,2.831116e-42,0.493932,3.108105e-80
3,0.588588,5.286829e-105,0.546338,7.629010e-88,-0.029632,3.226619e-01,0.556410,1.032644e-91,0.033885,2.580410e-01,0.040265,1.789008e-01,0.630494,1.011465e-124,0.587717,1.267628e-104,0.537591,1.370823e-84
4,0.314943,2.584306e-29,0.621592,1.766345e-130,0.536638,2.440424e-91,0.693344,2.157845e-174,0.585508,1.957125e-112,0.616875,5.492156e-128,0.418901,1.094179e-52,0.357062,9.340472e-38,0.662718,3.711341e-154
5,0.102940,4.066095e-04,0.319218,2.892342e-29,0.209070,4.415980e-13,0.467889,5.101944e-65,0.209734,3.711364e-13,0.200158,4.301384e-12,0.288208,6.258157e-24,0.039894,1.715726e-01,0.557874,3.728359e-97
6,0.238436,9.110234e-20,0.336148,9.017271e-39,-0.072836,6.088059e-03,0.455795,1.294312e-73,0.234713,3.463027e-19,0.172948,5.601673e-11,0.269252,5.862992e-25,0.248251,2.411456e-21,0.346786,2.598144e-41
7,0.078161,6.659047e-03,0.414102,4.396820e-51,0.249860,1.358649e-18,0.482536,3.113729e-71,0.432579,4.453566e-56,0.397208,8.782171e-47,0.235850,1.109427e-16,0.418481,3.070810e-52,0.403207,2.785117e-48
8,0.186618,6.653046e-12,0.535205,1.381290e-99,0.364831,3.345551e-43,0.667912,7.820275e-173,0.446400,3.207869e-66,0.490972,9.734817e-82,0.322622,1.218951e-33,0.074727,6.361310e-03,0.640082,1.969293e-154
9,0.412273,2.250036e-56,0.446212,7.752692e-67,0.128517,2.242396e-06,0.509714,6.737658e-90,0.213543,2.400828e-15,0.259575,3.638387e-22,0.518296,2.099632e-93,0.468222,2.760575e-74,0.586191,4.984614e-125


In [83]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_OT_InSeq_RSA_corr.csv')

### RSA of Repeats vs. InSeq in the Superior Temporal Gyrus

In [84]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [85]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_suptemp_inseq_repeat_pos3', 'p_suptemp_inseq_repeat_pos3', 'r_suptemp_inseq_repeat_pos4', 'p_suptemp_inseq_repeat_pos4', 'r_suptemp_inseq_repeat_pos5', 'p_suptemp_inseq_repeat_pos5', 'r_suptemp_inseq_pos3_repeat_pos4', 'p_suptemp_inseq_pos3_repeat_pos4', 'r_suptemp_inseq_pos3_repeat_pos5', 'p_suptemp_inseq_pos3_repeat_pos5', 'r_suptemp_inseq_pos4_repeat_pos5', 'p_suptemp_inseq_pos4_repeat_pos5', 'r_suptemp_inseq_pos4_repeat_pos3', 'p_suptemp_inseq_pos4_repeat_pos3', 'r_suptemp_inseq_pos5_repeat_pos3', 'p_suptemp_inseq_pos5_repeat_pos3', 'r_suptemp_inseq_pos5_repeat_pos4', 'p_suptemp_inseq_pos5_repeat_pos4']
final_df

,r_suptemp_inseq_repeat_pos3,p_suptemp_inseq_repeat_pos3,r_suptemp_inseq_repeat_pos4,p_suptemp_inseq_repeat_pos4,r_suptemp_inseq_repeat_pos5,p_suptemp_inseq_repeat_pos5,r_suptemp_inseq_pos3_repeat_pos4,p_suptemp_inseq_pos3_repeat_pos4,r_suptemp_inseq_pos3_repeat_pos5,p_suptemp_inseq_pos3_repeat_pos5,r_suptemp_inseq_pos4_repeat_pos5,p_suptemp_inseq_pos4_repeat_pos5,r_suptemp_inseq_pos4_repeat_pos3,p_suptemp_inseq_pos4_repeat_pos3,r_suptemp_inseq_pos5_repeat_pos3,p_suptemp_inseq_pos5_repeat_pos3,r_suptemp_inseq_pos5_repeat_pos4,p_suptemp_inseq_pos5_repeat_pos4
0,0.064506,2.843685e-02,-0.064202,2.919280e-02,0.373897,1.335435e-39,0.211993,3.423665e-13,0.048546,9.928681e-02,0.510307,1.584674e-77,0.354759,1.482640e-35,0.285015,5.226976e-23,0.066157,2.461400e-02
1,0.008749,7.488121e-01,0.204919,3.444062e-14,-0.009158,7.374987e-01,0.228010,2.752454e-17,0.250878,1.043209e-20,0.187556,4.330637e-12,-0.050030,6.692359e-02,-0.099844,2.489076e-04,0.284136,2.437720e-26
2,0.170333,7.530945e-10,0.290353,1.850707e-26,0.103707,1.916379e-04,0.323335,9.429166e-33,0.140992,3.725105e-07,0.368674,8.926286e-43,0.213422,9.605183e-15,0.037609,1.771982e-01,0.459190,3.193220e-68
3,0.279062,2.070212e-21,0.278345,2.641931e-21,0.138831,3.235901e-06,0.275789,6.265849e-21,0.274246,1.050354e-20,0.328637,1.625964e-29,0.278670,2.365087e-21,0.205806,3.865413e-12,0.482789,3.144513e-66
4,0.154331,6.675266e-08,-0.182904,1.406411e-10,-0.011458,6.902755e-01,-0.189740,2.750457e-11,-0.114867,6.120490e-05,0.035232,2.203149e-01,0.212264,8.182630e-14,0.257405,8.556893e-20,-0.131779,4.162564e-06
5,0.141764,1.055560e-06,0.011245,7.000686e-01,-0.147031,4.099069e-07,0.182778,2.709233e-10,0.016244,5.778798e-01,0.030476,2.963736e-01,0.247341,7.494673e-18,0.234387,3.857422e-16,0.265879,1.767788e-20
6,0.289936,7.602296e-29,0.391471,4.159240e-53,0.023881,3.690293e-01,0.432822,8.938830e-66,0.359674,1.593852e-44,0.277779,1.605927e-26,0.330486,1.850052e-37,0.285375,5.842569e-28,0.454820,2.863750e-73
7,0.102221,3.815089e-04,0.070840,1.394769e-02,-0.004722,8.699839e-01,0.018212,5.278227e-01,0.047324,1.007373e-01,0.102034,3.910800e-04,0.227629,1.291464e-15,0.087200,2.458775e-03,0.091017,1.569915e-03
8,-0.410804,2.201874e-55,0.580201,1.100905e-120,0.448743,5.576018e-67,0.671153,4.161138e-175,0.503723,1.287897e-86,0.636934,1.806217e-152,-0.348933,1.993442e-39,-0.324353,5.277200e-34,0.644744,2.217123e-157
9,0.266580,2.476716e-23,0.332633,3.906098e-36,0.278235,2.365096e-25,0.339207,1.350338e-37,0.280967,7.692206e-26,0.328357,3.337108e-35,0.267165,1.971816e-23,0.142304,1.582469e-07,0.475708,6.035763e-77


In [86]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_repeat_InSeq_RSA_corr.csv')

### RSA of Skips vs. InSeq in the Superior Temporal Gyrus

In [87]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [88]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_suptemp_inseq_skip_pos3', 'p_suptemp_inseq_skip_pos3', 'r_suptemp_inseq_skip_pos4', 'p_suptemp_inseq_skip_pos4', 'r_suptemp_inseq_skip_pos5', 'p_suptemp_inseq_skip_pos5', 'r_suptemp_inseq_pos3_skip_pos4', 'p_suptemp_inseq_pos3_skip_pos4', 'r_suptemp_inseq_pos3_skip_pos5', 'p_suptemp_inseq_pos3_skip_pos5', 'r_suptemp_inseq_pos4_skip_pos5', 'p_suptemp_inseq_pos4_skip_pos5', 'r_suptemp_inseq_pos4_skip_pos3', 'p_suptemp_inseq_pos4_skip_pos3', 'r_suptemp_inseq_pos5_skip_pos3', 'p_suptemp_inseq_pos5_skip_pos3', 'r_suptemp_inseq_pos5_skip_pos4', 'p__suptemp_inseq_pos5_skip_pos4']
final_df

,r_suptemp_inseq_skip_pos3,p_suptemp_inseq_skip_pos3,r_suptemp_inseq_skip_pos4,p_suptemp_inseq_skip_pos4,r_suptemp_inseq_skip_pos5,p_suptemp_inseq_skip_pos5,r_suptemp_inseq_pos3_skip_pos4,p_suptemp_inseq_pos3_skip_pos4,r_suptemp_inseq_pos3_skip_pos5,p_suptemp_inseq_pos3_skip_pos5,r_suptemp_inseq_pos4_skip_pos5,p_suptemp_inseq_pos4_skip_pos5,r_suptemp_inseq_pos4_skip_pos3,p_suptemp_inseq_pos4_skip_pos3,r_suptemp_inseq_pos5_skip_pos3,p_suptemp_inseq_pos5_skip_pos3,r_suptemp_inseq_pos5_skip_pos4,p__suptemp_inseq_pos5_skip_pos4
0,0.224330,1.249913e-14,0.210827,4.633716e-13,0.008837,7.642622e-01,0.488230,3.685513e-70,0.059234,4.424225e-02,0.100234,6.498535e-04,0.167998,9.375190e-09,0.094668,1.283364e-03,0.238160,2.405133e-16
1,0.099802,2.504358e-04,0.224067,9.857203e-17,-0.007413,7.861541e-01,0.309247,3.942424e-31,0.003758,8.906132e-01,0.041243,1.310126e-01,0.124758,4.562686e-06,0.009956,7.155556e-01,0.143123,1.399484e-07
2,0.092058,9.364398e-04,0.132221,1.903531e-06,0.130567,2.559947e-06,0.239425,2.914920e-18,-0.012417,6.560293e-01,0.211556,1.653356e-14,0.419579,4.035808e-56,0.374663,3.163132e-44,0.345862,1.587497e-37
3,0.486581,2.148097e-67,0.639839,1.530141e-129,0.324836,7.741339e-29,0.647898,7.781267e-134,0.290339,4.062188e-23,0.306414,1.092144e-25,0.529188,1.495391e-81,0.647966,7.148710e-134,0.748916,2.661935e-201
4,0.299447,1.571791e-26,0.519445,1.042398e-84,-0.096454,7.729015e-04,0.547089,1.465270e-95,-0.082760,3.936989e-03,0.015168,5.978107e-01,0.494479,9.860269e-76,0.382323,1.800031e-43,0.629795,6.436865e-135
5,-0.215799,7.387008e-14,0.069286,1.748532e-02,0.199239,5.408203e-12,0.214003,1.197448e-13,0.136001,2.859634e-06,0.372102,6.322096e-40,0.033438,2.518825e-01,-0.068621,1.859822e-02,0.284830,2.182994e-23
6,0.514348,1.581079e-96,0.489729,2.388441e-86,0.191213,3.912641e-13,0.518201,3.383528e-98,0.123140,3.335638e-06,0.157004,2.819686e-09,0.570785,2.750874e-123,0.442685,4.563790e-69,0.596532,2.561016e-137
7,0.077119,7.425744e-03,-0.017870,5.356096e-01,-0.177540,5.527052e-10,0.145896,3.688717e-07,0.046597,1.060826e-01,-0.038977,1.765190e-01,0.131821,4.447152e-06,0.043247,1.336763e-01,0.110132,1.286030e-04
8,0.233833,5.322036e-18,0.389100,2.167074e-49,0.358762,9.797756e-42,0.450564,1.418408e-67,0.390834,7.471127e-50,0.507093,6.104634e-88,0.424084,2.825259e-59,0.231888,1.014533e-17,0.506340,1.209915e-87
9,0.155609,9.543139e-09,0.266067,3.023173e-23,0.084790,1.849040e-03,0.298581,4.050433e-29,0.083737,2.107365e-03,0.146504,6.698117e-08,0.313946,3.587622e-32,0.128022,2.454295e-06,0.362531,4.502813e-43


In [89]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_skip_InSeq_RSA_corr.csv')

# The end